In [1]:
import numpy as np
from coverage import *
from datetime import datetime, timezone, timedelta

from shapely.geometry import Point
import geopandas as gpd
import pandas as pd
import branca
import folium

! mkdir tmp

mkdir: tmp: File exists


In [2]:
from landsat import Instrument, Platform, Scene

In [3]:
ts = load.timescale()
now_utc = datetime.now(timezone.utc)
tom_utc = now_utc + timedelta(days=1, hours=2)

now_ts = ts.from_datetime(now_utc)
tom_ts = ts.from_datetime(tom_utc)

In [4]:
now_utc

datetime.datetime(2022, 6, 15, 21, 14, 29, 72810, tzinfo=datetime.timezone.utc)

In [5]:
num_days = 1
# tles = gen_sats(sat_nos=[39084,49260])

tles = gen_sats(
    # sat_nos=[Platform.norad_id]
    sat_nos=[39084,49260]
)

inst = camera_model(
    name="tirs", 
    fl=Instrument.focal_length_mm, 
    pitch=Instrument.pitch_um*1e-3, 
    h_pix=Instrument.rows, 
    v_pix=Instrument.cols, 
)

times = gen_times(
    start_yr=2022, 
    start_mo=6, 
    start_day=10, 
    days=num_days, 
    step_min=Instrument.img_period
    ) # Adjusted TIRS for 1 min FOVs

xcell_size = ycell_size = .1


Satellite(s) Loaded from TLE:
[<EarthSatellite LANDSAT 8 catalog #39084 epoch 2022-06-14 19:33:45 UTC>]
[<EarthSatellite LANDSAT 9 catalog #49260 epoch 2022-06-14 10:29:20 UTC>]
Propogation time: 
 2022-06-10 00:00:00+00:00 
to 
 2022-06-10 23:59:54+00:00


In [7]:
## AOI selection

wrs2 = gpd.read_file('./WRS2_descending_0/WRS2_descending.shp')

## Select 
# from gpd naturalearth dataset (filter by .name for country, .continent for continent)
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))

## Filter by AOI - should pass to forecast function instead?
# aoi =  world[world.name == "Brazil"].geometry
# aoi =  world[world.continent == "South America"].geometry
# aoi = world[world.name == "United States of America"].geometry

aoi = gpd.read_file('./aois/eastern_us.geojson').geometry
aoi

xmin, ymin, xmax, ymax= aoi.total_bounds
wrs2 = wrs2.cx[xmin: xmax, ymin: ymax]


In [8]:
# wrs2.explore()

In [9]:
# ssp_df.geometry.explore()

In [17]:
## Batch FOV generation over N satellites
gdfs = []
for tle in tles:
    sat = tle[0]
    poly_df = forecast_fovs(sat, times, inst)
    gdfs.append(poly_df)

poly_df = gpd.GeoDataFrame(pd.concat(gdfs, ignore_index=True), crs="epsg:4326")

## Filter shapes crossing anti-meridian - also in main function
## TODO: Switch to stactools solution
poly_df["lonspan"] = poly_df.bounds['maxx'] - poly_df.bounds['minx']

## Create cmap for unique satellites and create color column
sat_ids = list(poly_df["id"].unique()).sort()
cmap = branca.colormap.StepColormap(['red', 'blue'], sat_ids, vmin=139084, vmax = 149260)

poly_df['color'] = poly_df['id'].apply(cmap)

plot_df = poly_df[poly_df["lonspan"] < 20].copy()

for satname in plot_df.satellite.unique():
    plot_df[plot_df.satellite==satname].to_file("./tmp/{}_fovs.geojson".format(satname.replace(" ", "_")))
plot_df

,geometry,satellite,id,time,lonspan,color
100,"POLYGON ((-124.50340 81.00725, -124.14768 82.6...",LANDSAT 8,139084,2022-06-10 00:36:40 UTC,13.056827,#ff0000ff
101,"POLYGON ((-133.07357 80.94444, -134.54291 82.5...",LANDSAT 8,139084,2022-06-10 00:37:02 UTC,13.800111,#ff0000ff
102,"POLYGON ((-141.35621 80.69119, -144.43125 82.2...",LANDSAT 8,139084,2022-06-10 00:37:24 UTC,14.143407,#ff0000ff
103,"POLYGON ((-149.06618 80.26231, -153.35458 81.7...",LANDSAT 8,139084,2022-06-10 00:37:46 UTC,13.874563,#ff0000ff
104,"POLYGON ((-156.03211 79.67966, -161.10836 81.0...",LANDSAT 8,139084,2022-06-10 00:38:08 UTC,13.209518,#ff0000ff
...,...,...,...,...,...,...
7797,"POLYGON ((111.98320 -79.91424, 106.23565 -78.6...",LANDSAT 9,149260,2022-06-10 23:38:38 UTC,13.399840,#0000ffff
7798,"POLYGON ((105.86540 -80.76655, 100.42456 -79.3...",LANDSAT 9,149260,2022-06-10 23:39:00 UTC,14.113241,#0000ffff
7799,"POLYGON ((98.63112 -81.49956, 93.82453 -80.019...",LANDSAT 9,149260,2022-06-10 23:39:22 UTC,14.455009,#0000ffff
7800,"POLYGON ((90.21578 -82.08009, 86.45600 -80.510...",LANDSAT 9,149260,2022-06-10 23:39:44 UTC,14.206471,#0000ffff


In [11]:
## Select AOI from gpd naturalearth dataset (filter by .name for country, .continent for continent)
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))

## Filter by AOI - should pass to forecast function instead?
# aoi =  world[world.name == "Brazil"].geometry
# aoi =  world[world.continent == "South America"].geometry
aoi = world[world.name == "United States of America"]#.geometry

aoi = gpd.read_file('./aois/eastern_us.geojson').geometry
aoi

xmin, ymin, xmax, ymax= aoi.total_bounds
plot_df = plot_df.cx[xmin: xmax, ymin: ymax]

In [12]:
## Coverage data analysis for single satellite/ batch of satellites

# 1) Create a grid of equally spaced points
grid, grid_shape = create_grid(aoi.total_bounds, xcell_size, ycell_size)

# 2) Add "n_visits" column to grid using sjoin/ dissolve
shapes = gpd.GeoDataFrame(plot_df.geometry)
merged = gpd.sjoin(shapes, grid, how='left', predicate="intersects")
merged['n_visits']=0 # this will be replaced with nan or positive int where n_visits > 0
dissolve = merged.dissolve(by="index_right", aggfunc="count") # no difference in count vs. sum here?
grid.loc[dissolve.index, 'n_visits'] = dissolve.n_visits.values

grid.to_file('./tmp/n_visits.geojson')
grid.n_visits.fillna(0).describe()

count    113250.000000
mean          0.230031
std           0.516558
min           0.000000
25%           0.000000
50%           0.000000
75%           0.000000
max           2.000000
Name: n_visits, dtype: float64

In [13]:
## Plotting FOVs

## Make a folium map
m = plot_df.explore(color="color", tooltip=["satellite", "time"])

## Add WRS2
# folium.GeoJson(data=wrs2["geometry"], overlay=False).add_to(m)


## View or save
m.save("./tmp/fov_output.html")

In [14]:
## Plotting Revisit Map

## Form 2D array of n_visits based on grid shape
img = np.rot90(grid.n_visits.values.reshape(grid_shape))

## Create colormap and apply to img
## TODO: Make this our geoTIFF item for STAC catalog
colormap = branca.colormap.step.viridis.scale(1, grid.n_visits.max())
# colormap = branca.colormap.step.viridis.scale(1, 2)

def colorfunc(x):
    if np.isnan(x):
        return (0,0,0,0)
    else:
        return colormap.rgba_bytes_tuple(x)

# Apply cmap to img array and rearrange for RGBA
cmap = np.vectorize(colorfunc)
rgba_img = np.array(cmap(img))
rgba_img = np.moveaxis(rgba_img, 0, 2)

# Update image corner bounds based on cell size
xmin, ymin, xmax, ymax= grid.total_bounds
xmin = xmin - xcell_size/2
ymin = ymin - ycell_size/2
xmax = xmax + xcell_size/2
ymax = ymax + ycell_size/2

m = folium.Map()
m.fit_bounds([[ymin, xmin], [ymax, xmax]])
m.add_child(folium.raster_layers.ImageOverlay(rgba_img, opacity=.4, mercator_project=True,# crs="EPSG:4326",
                                 bounds = [[ymin,xmin],[ymax,xmax]]))
colormap.add_to(m)
m.save("./tmp/coverage_output.html")